In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
a = 2
b = 5
c = 1
d = 1

def x_dot(x, y, omega):
    dxdt = a*omega - (b + d) * x + c/(omega**2) * x**2 * y

def y_dot(x, y, omega):
    dydt = b*x - c/(omega**2) * x**2 * y
     